In [1]:
import PyPDF2
import itertools
import spacy
import networkx
from pyvis.network import Network

import pandas as pd
import matplotlib.pyplot as plt
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import math
import re
import seaborn as sns
import random
import numpy as np

In [2]:
graphLocation = 'C:/Users/mj514/Documents/blogdownBlog/blog/static/html/'

In [3]:
mueller1 = open('Mueller-Report.pdf', 'rb') 

In [4]:
mueller1Reader = PyPDF2.PdfFileReader(mueller1)

In [5]:
muellerPages = []
for pageNum in range(mueller1Reader.numPages):
    muellerPages.append([par for par in mueller1Reader.getPage(pageNum).extractText().split('\n')])
mueller1.close()

In [6]:
muellerParagraphs = sum(muellerPages,[]) #flatten the list of lists

In [7]:
len(muellerParagraphs)

897

In [8]:
nlp = spacy.load('C:/Users/mj514/Anaconda3/lib/site-packages/en_core_web_lg/en_core_web_lg-2.0.0')

In [49]:
#From spacy documentation
class EntityMatcher(object):
    name = "entity_matcher"

    def __init__(self, nlp, terms, label):
        patterns = [nlp.make_doc(text) for text in terms]
        self.matcher = PhraseMatcher(nlp.vocab)
        self.matcher.add(label, None, *patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        for match_id, start, end in matches:
            span = Span(doc, start, end, label=match_id)
            doc.ents = list(doc.ents) + [span]
        return doc

terms = (u"collude", u"collusion", 
         u"conspiracy", u"conspire",u"hack", u"hacking", u"cyber intrusions", 
         u"russian hacking", u"hackers",u"social media",u"computer intrusions",
         u"cybersecurity",u"emails","email")
concept_matcher = EntityMatcher(nlp, terms, "THEME")

nlp.add_pipe(concept_matcher, after="ner")

print(nlp.pipe_names)  # The components in the pipeline

doc = nlp(u"This is a hacking, collusion, text about Barack Obama who is the Presedent of The United States")
print([(ent.text, ent.label_) for ent in doc.ents])

ValueError: [E007] 'entity_matcher' already exists in pipeline. Existing names: ['tagger', 'parser', 'ner', 'entity_matcher']

In [10]:
%time
nlpParagraphs= list(nlp.pipe(muellerParagraphs, n_threads=16, batch_size=50))

In [11]:
nlpParagraphs[101]

mails belonging to_, and emails from 2015 relating to Republican Party employees (under the portfolio name "The United States Republican Party"). "The United States Republican Party" portfolio contained approximately 300 emails from a variety of GOP members, PACs, campaigns, state parties, and businesses dated between May and October 2015. According to open-source reporting, these victims shared the same 41 

In [12]:
from collections import Counter
aDoc = nlpParagraphs[101]

entCountDict = Counter([(ent.text, ent.label_) for ent in doc.ents if not re.match(r"\.", ent.text) and (ent.label_=='GPE' or ent.label_ =='PERSON' or ent.label_ =='THEME') ])

for k,v in entCountDict.items():
    print(f"Label: {k[1]} text:{k[0]} count:{v}")

Label: THEME text:hacking count:1
Label: THEME text:collusion count:1
Label: PERSON text:Barack Obama count:1
Label: PERSON text:Presedent count:1
Label: GPE text:The United States count:1


# Tried using fuzzy wuzzy library to dedup
Wasn't happy with results so I don't use it, but feel free to try it out.

In [12]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [13]:

ents = [[ent.text.strip() for ent in doc.ents if not re.match(r"\.", ent.text) 
         and (ent.label_=='GPE' or ent.label_ =='PERSON' or ent.label_ =='THEME')] for doc in nlpParagraphs]
distinctEnts = list(set(sum(ents,[])))

In [ ]:
dedupDict = {}
for i in range(len(distinctEnts)):
    if(i%10 == 0):
        print(len(distinctEnts))
    searchTerm = distinctEnts.pop(0)
    matchList = process.extractBests(searchTerm, distinctEnts[1:],score_cutoff=90)
    
    if(len(matchList) == 0):
        dedupDict[searchTerm] = searchTerm
    else:
        print(searchTerm)
        print(matchList)
        for m in matchList:
            dedupDict[m[0]] = searchTerm
            distinctEnts.remove(m[0])

In [14]:
matchList[0][0]

NameError: name 'matchList' is not defined

In [13]:
def from_nx(graphVizNet, nx_graph):
        """
        This method takes an exisitng Networkx graph and translates
        it to a PyVis graph format that can be accepted by the VisJs
        API in the Jinja2 template. This operation is done in place.

        :param nx_graph: The Networkx graph object that is to be translated.
        :type nx_graph: networkx.Graph instance
        >>> nx_graph = Networkx.cycle_graph()
        >>> nt = Network("500px", "500px")
        # populates the nodes and edges data structures
        >>> nt.from_nx(nx_graph)
        >>> nt.show("nx.html")
        """
        assert(isinstance(nx_graph, networkx.Graph))
        edges = nx_graph.edges(data=True)
        nodes = nx_graph.nodes()
        if len(edges) > 0:
            for e in edges:
                graphVizNet.add_node(e[0], e[0], title=str(e[0]), color = nx_graph.node[e[0]]['color'], value = nx_graph.node[e[0]]['value'])
                graphVizNet.add_node(e[1], e[1], title=str(e[1]), color = nx_graph.node[e[1]]['color'], value = nx_graph.node[e[1]]['value'])
                graphVizNet.add_edge(e[0], e[1], value = nx_graph.edges[e[0],e[1]]['weight'])
        else:
            graphVizNet.add_nodes(nodes)
        return graphVizNet

In [14]:
import palettable
# selectedColors = palettable.colorbrewer.sequential.Blues_4.hex_colors
selectedColors = palettable.cartocolors.diverging.Earth_7.hex_colors[-5:]
selectedColors

['#D6BD8D', '#EDEAC2', '#B5C8B8', '#79A7AC', '#2887A1']

In [17]:
numParagraphs = len(nlpParagraphs)
minConnections = 3

G = networkx.Graph()
G.clear()

colorMap = {'GPE':'Blue','PERSON':'Red','ORG':'Green','THEME':'Black'}
#nodeTypes = ['GPE','PERSON','ORG','THEME']
#colorMap = {nType:selectedColors[i] for i,nType in enumerate(nodeTypes)}
print(colorMap)
entCount = 0
for parNum, doc in enumerate(nlpParagraphs[0:numParagraphs]):
    G.add_node(f"par_{parNum}", t = 'Paragraph', color = 'grey')
    entCount = 0
    
    entCountDict = Counter([(ent.string.strip(),ent.label_) for ent in doc.ents if not bool(re.search(r":|\.", ent.text))  and (ent.label_=='GPE' or ent.label_ =='PERSON' or ent.label_ =='THEME')])

    for ent,cnt in entCountDict.items():
        G.add_node(ent[0],label = ent[1], color = colorMap[ent[1]])
        G.add_edge(ent[0],f"par_{parNum}", weight = cnt)#math.log(cnt + 1))
    if len(entCountDict) <minConnections:
        G.remove_node(f"par_{parNum}")
print(G.number_of_nodes())       
nodesToRemove = []
for node in G.nodes():
    nodeDegree = G.degree(node)
    if nodeDegree>0:
        G.node[node]['value'] = nodeDegree
    else:
        nodesToRemove.append(node)

for node in nodesToRemove:
    G.remove_node(node)

{'GPE': 'Blue', 'PERSON': 'Red', 'ORG': 'Green', 'THEME': 'Black'}
2606


In [18]:
G.number_of_nodes()

2554

In [19]:
pyvisG = Network("1000px", "1000px")
from_nx(pyvisG,G)
pyvisG.show_buttons(filter_=['physics'])
pyvisG.show('fullMuller.html')

In [20]:
pyvisG.save_graph(graphLocation + 'fullMullerGraph.html')

In [21]:
exampleGraph = Network("400px", "400px", notebook = 'True')
exampleGraph.add_node('Paragraph', color = 'Grey')
exampleGraph.add_node('Location', color = 'Blue')
exampleGraph.add_node('Person', color = 'Red')
exampleGraph.add_node('Concept/Theme', color = 'Black')

exampleGraph.add_edge('Paragraph','Location')
exampleGraph.add_edge('Paragraph','Person')
exampleGraph.add_edge('Paragraph','Concept/Theme')

#pyvisG.show_buttons(filter_=['physics'])
exampleGraph.save_graph(graphLocation + 'setupGraph.html')
exampleGraph.show('setupGraph.html')

In [20]:
d = networkx.algorithms.pagerank(G)
print(d.__class__)
pageRank = pd.DataFrame([d]).T

<class 'dict'>


In [21]:
pageRank.sort_values(0, ascending = False).head(10)

,0
Russia,0.030103
Cohen,0.022387
Corney,0.021937
Flynn,0.020361
United States,0.012180
Papadopoulos,0.011256
McGahn,0.011012
Manafort,0.009693
Clinton,0.009310
Trump,0.008640


In [42]:
def getFirstOrderGraph(graph, node):
    nodes = []
    [nodes.append(n) for n in graph.neighbors(node)]
    nodes += [node]
    subG = graph.subgraph(nodes)
    print(f"Nodes: {subG.number_of_nodes()}")
    print(f"Edges: {subG.number_of_edges()}")
    print(f"Self Loops: {subG.number_of_selfloops()}")
    return subG
def getSecondOrderSubgraph(graph, node):
    nodes = []
    for neighbor_list in [graph.neighbors(n) for n in graph.neighbors(node)]:
        for n in neighbor_list:
            nodes.append(n)
    nodes += [n for n in graph.neighbors(node)]
    subG = graph.subgraph(nodes)
    print(f"Nodes: {subG.number_of_nodes()}")
    print(f"Edges: {subG.number_of_edges()}")
    print(f"Self Loops: {subG.number_of_selfloops()}")
    return subG

In [47]:
subGraph = getFirstOrderGraph(G,'par_259')
pyvisG = Network("500px", "500px")
from_nx(pyvisG,subGraph)
#pyvisG.show_buttons(filter_=['physics'])
pyvisG.show('testSubgraph.html')
#pyvisG.show('C:/Users/mj514/Documents/blogdownBlog/blog/content/page/test.html')

Nodes: 7
Edges: 6
Self Loops: 0


In [44]:
subGraph

In [48]:
muellerParagraphs[259]

'the DNC hacking announcement to the June 9 OSC-KA V _00029 (6/14/16 Email, Goldstone to E. Agalarov & Kaveladze (10:09 a.m.)). The investigation did not identify evidence connecting the events of June 9 to the GRU\'s hack-and-dump operation. OSC-KA V _00029-30 (6/14/16 Email, Goldstone to E. Agalarov). 746 747 Kaveladze 11/16/17 302, at 8; Call Records ofike. Kaveladze 748 Kaveladze 11/16/17 302, at 8; Call Records of Ike Kaveladze On June 14, 2016 Kaveladze\'s teenage daughter emailed asking how the June 9 meeting had gone, and Kaveladze responded, "meeting was boring. The Russians did not have an bad info on Hilar " KA V _00257 (6/14/16 Email, I. Kaveladze to A. Kaveladze; 749 Goldstone 2/8/18 302, '

# Node2Vec

In [81]:
nxG = G

startingNodes = nxG.nodes()
numWalks = 100000

walkLengths = np.random.normal(10, 3, numWalks).astype(int)
walkLengths[walkLengths < 0] = 3 # has to do three walks

walks = []
for walkLen in walkLengths:
    start = random.choice(list(startingNodes))
    if nxG.degree(start) > 1: #Check if the starting node has neighbors
        walk = [start]

        for step in range(walkLen):
            #print(f"onNode: {walk}")
            paths = [n for n in nxG.neighbors(walk[-1])]
            #print(paths)
            walk.append(random.choice(paths))
        walks.append(walk)

In [149]:
randomWalk = G.subgraph(walks[3])
randomVis = Network("500px", "500px", notebook=True)
from_nx(randomVis,randomWalk)
#pyvisG.show_buttons(filter_=['physics'])
randomVis.show('randomWalk.html')

In [152]:
randomWalk2 = G.subgraph(walks[6000])
randomVis2 = Network("500px", "500px", notebook=True)
from_nx(randomVis2,randomWalk2)
#pyvisG.show_buttons(filter_=['physics'])
randomVis2.show('randomWalk2.html')

In [221]:
from gensim.models import Word2Vec
w2vModel = Word2Vec(walks, size=300, window=10, min_count=1, workers=16)

In [222]:
w2vModel.wv['par_390']

array([ 0.92456126,  0.9449051 , -0.94948184, -0.4889605 , -0.98223823,
        0.8853492 , -0.11918477, -0.66409695, -0.12330315,  0.8563422 ,
       -0.3176424 , -0.58984476,  0.05426842,  0.70515704, -0.6734733 ,
       -0.3863421 , -0.4364722 , -0.3857968 ,  0.22750401,  0.0111002 ,
       -0.13708027, -0.5119472 , -1.6217746 ,  0.29460165, -0.03582903,
       -0.6424805 ,  0.10051477,  0.93095225,  1.2485225 ,  0.3563679 ,
       -1.0603137 ,  0.7071692 , -0.07872218,  0.00490348,  0.17187384,
        0.67104197, -0.83084995, -0.9543589 ,  0.21825725,  1.3602228 ,
       -0.36175582,  0.11262723, -0.10977689,  0.10455734, -0.85053694,
       -0.07955333, -0.46475342, -0.6362551 ,  0.43668327,  0.2287611 ,
       -0.01633505,  0.29489136,  0.36975288, -0.19889747, -1.9386314 ,
       -0.2866822 , -0.6811361 , -1.0950114 ,  0.74643904, -0.37689385,
       -1.3955669 ,  0.19111334,  0.9709352 ,  0.23986061, -0.34570584,
        0.19158393,  0.72093254, -0.6905862 , -0.23265459, -0.36

In [223]:
w2vModel.wv.most_similar('par_390')

[('par_189', 0.7216188311576843),
 ('par_25', 0.7195272445678711),
 ('par_190', 0.6681944131851196),
 ('Olga Polonskaya', 0.6547738909721375),
 ('Ivan Timofeev', 0.6297422051429749),
 ('par_194', 0.611091136932373),
 ('par_838', 0.5883481502532959),
 ('par_208', 0.5767848491668701),
 ('par_23', 0.5747522115707397),
 ('par_193', 0.5183202028274536)]

In [224]:
w2vModel.most_similar(positive=['Russia', 'the District of Columbia'], negative=['Moscow'])

[('Virginia', 0.3910454511642456),
 ('Turkey', 0.3301968574523926),
 ('Concord', 0.3220016360282898),
 ('Transcript', 0.3180886507034302),
 ('Manafo1', 0.30232226848602295),
 ('par_356', 0.30222442746162415),
 ('Manafort', 0.30213433504104614),
 ('par_355', 0.30133575201034546),
 ('conspiracy', 0.2977093756198883),
 ('par_890', 0.2973538637161255)]

In [219]:
w2vModel.most_similar(positive=['Russia', 'the United States'], negative=['Putin'])

[('conspiracy', 0.5141558647155762),
 ('Kilimnik', 0.3893536627292633),
 ('Ukraine', 0.378567099571228),
 ('social media', 0.37721455097198486),
 ('Carter Page', 0.36491039395332336),
 ('par_367', 0.3607674539089203),
 ('hacking', 0.3286607563495636),
 ('Oleg Deripaska', 0.3231571316719055),
 ('the District of Columbia', 0.3220616579055786),
 ('Flynn', 0.31990569829940796)]

In [92]:
w2vModel.wv.most_similar('par_390')

[('par_25', 0.7742655277252197),
 ('Olga Polonskaya', 0.7491888999938965),
 ('par_189', 0.7278563976287842),
 ('par_190', 0.661663293838501),
 ('Ivan Timofeev', 0.6461209058761597),
 ('par_208', 0.6244201064109802),
 ('par_23', 0.6139870882034302),
 ('par_194', 0.6036626696586609),
 ('par_94', 0.5973951816558838),
 ('par_838', 0.5720088481903076)]

In [61]:
muellerParagraphs[390]

'Campaign through the anonymous release of information damaging to candidate Clinton. On January 27, 2017, Papadopoulos agreed to be interviewed by FBI agents, who informed him that the interview was part of the investigation into potential Russian government interference in the 2016 presidential election. During the interview, Papadopoulos lied about the timing, extent, and nature of his communications with Joseph Mifsud, Olga Polonskaya, and Ivan Timofeev. With respect to timing, Papadopoulos acknowledged that he had met Mifsud and that Mifsud told him the Russians had "dirt" on Clinton in the form of "thousands of emails." But Papadopoulos stated multiple times that those communications occurred before he joined the Trump Campaign and that it was a "very strange coincidence" to be told of the "dirt" before he started working for the Campaign. This account was false. Papadopoulos met Mifsud for the first time on approximately March 14, 2016, after Papadopoulos had already learned he 

In [93]:
muellerParagraphs[189]

'U.S. Department of Justice Atief\'Hey \\l/erk Pretittet // Ma;· CeHtaiH Material Preteeteti UHtier Fed. R. CFiffl. P. 6(e) 2016 to early October 2016.394 In late April 2016, Papadopoulos was told by London-based professor Joseph Mifsud, immediately after Mifsud\'s return from a trip to Moscow, that the Russian government had obtained "dirt" on candidate Clinton in the form of thousands of emails. One week later, on May 6, 2016, Papadopoulos suggested to a representative of a foreign government that the Trump Campaign had received indications from the Russian government that it could assist the Campaign through the anonymous release of information that would be damaging to candidate Clinton. Papadopoulos shared information about Russian "dirt" with people outside of the Campaign, and the Office investigated whether he also provided it to a Campaign official. Papadopoulos and the Campaign officials with whom he interacted told the Office that they did · not recall that Papadopoulos pass

In [186]:
paragraphDF = pd.DataFrame({'text':muellerParagraphs,'paragraph':[f'par_{num}' for num in range(len(muellerParagraphs))]})

In [194]:
par = 'par_390'

def showMoreLikeThis(par):
    mostSimilar = w2vModel.wv.most_similar(par, topn=20)

    mostSimilarParagraphs = [simTuple for simTuple in mostSimilar if simTuple[0].startswith('par')]
    similarDF = pd.DataFrame(mostSimilarParagraphs, columns=['paragraph','similarity'])
    showDF = similarDF.merge(paragraphDF, on = 'paragraph')
    pd.options.display.max_colwidth = -1
    return showDF

In [220]:
showMoreLikeThis('collusion')

,paragraph,similarity,text
0,par_17,0.741251,"statements about the results of our investigation and the Special Counsel's charging decisions, and we then provide an overview of the two volumes of our report. The report describes actions and events that the Special Counsel's Office found to be supp01ted by the evidence collected in our investigation. In some instances, the report points out the absence of evidence or conflicts in the evidence about a particular fact or event. In other instances, when substantial, credible evidence enabled the Office to reach a conclusion with confidence, the report states that the investigation established that certain actions or events occurred. A statement that the investigation did not establish particular facts does not mean there was no evidence of those facts. In evaluating whether evidence about collective action of multiple individuals constituted a crime, we applied the framework of conspiracy law, not the concept of ""collusion."" In so doing, the Office recognized that the word ""collud[ e ]"" was used in communications with the Acting Attorney General confirming certain aspects of the investigation's scope and that the term has frequently been invoked in public reporting about the investigation. But collusion is not a specific offense or theory of liability found in the United States Code, nor is it a term of art in federal criminal law. For those reasons, the Office's focus in analyzing questions of joint criminal liability was on conspiracy as defined in federal law. In connection with that analysis, we addressed the factual question whether members of the Trump Campaign ""coordinat[ ed]""-a term that appears in the appointment order-with Russian election interference activities. Like collusion, ""coordination"" does not have a settled definition in federal criminal law. We understood coordination to require an agreement-tacit or express-between the Trump Campaign and the Russian government on election interference. That requires more than the two parties taking actions that were informed by or responsive to the other's actions or interests. We applied the term coordination in that sense when stating in the report that the investigation did not establish that the Trump Campaign coordinated with the Russian government in its election interference activities. * * *"
1,par_594,0.707229,".m., the President called McGahn on McGahn' s personal cell phone and they spoke for about l 5 minutes.564 McGahn did not have a clear memory of the call but thought they might have discussed the stories reporting that the President was under investigation.565 Beginning early the next day, June 15, 2017, the President issued a series of tweets acknowledging the existence of the obstruction investigation and criticizing it. He wrote: ""They made up a phony collusion with the Russians story, found zero proof, so now they go for obstruction of justice on the phony story. Nice"";566 ""You are witnessing the single greatest WITCH HUNT in American political history-led by some very bad and conflicted people!"";567 and ""Crooked H destroyed phones w/ hammer, 'bleached' emails, & had husband meet w/AG days 560 Special Counsel's Office Attorney 6/15/17 Notes. 561 Devlin Barrett et al., Special counsel is investigating Trump for possible obstruction of justice, officials say, Washington Post (June 14, 2017). 562 CNN, for example, began running a chyron at 6:55 p.m. that stated: ""WASH POST: MUELLER INVESTIGATING TRUMP FOR OBSTRUCTION OF JUSTICE."" CNN, (June 14, 2017, published online at 7: 15 p.m. ET). 563 Devlin Barrett et al., Special counsel is investigating Trump for possible obstruction of justice, officials say, Washington Post (June 14, 2017). 564 SCR026_000183 (President's Daily Diary, 6/14/17) (reflecting call from the President to McGahn on 6/14/17 with start time 10:31 p.m. and end time 10:46 p.m.); Call Records of Don McGahn. 565 McGahn 2/28/ 19 302, at 1-2. McGahn thought he and the President also

In [236]:
pd.DataFrame([networkx.algorithms.pagerank(G)]).T.sort_values(0,ascending = False)

,0
Russia,0.028861
Cohen,0.021626
Corney,0.021351
Flynn,0.019705
United States,0.011874
McGahn,0.010744
Papadopoulos,0.010675
emails,0.009447
Manafort,0.009304
email,0.008997
